In [22]:
!pip install mnist

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog 
from sklearn.metrics import accuracy_score
import random
import sys
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.svm import SVC
import math
from sklearn.preprocessing import StandardScaler
import cvxpy as cp
import sys
from random import seed
np.random.seed(1)
seed(1)
import time
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from sklearn import svm
import math
import cvxopt
import mnist

In [5]:
def predict(X, w, b):
  return np.sign(inner_prod(w, X)+b).reshape(-1,1)
def accuracy(a,b):
  n=0
  for i in range(len(a)):
    if(a[i]==b[i]):
      n=n+1
  return n/len(a)
def getHyperPlaneFromTwoPoints(x1, x2):
  assert(x1.shape[0]==x2.shape[0])
  d = x1.shape[0]
  w = (2 + 1e-8) * (x2 - x1) / (np.linalg.norm(x1 - x2) ** 2)
  b = - np.dot(w , (0.5 * (x1 + x2)))
  return w, b

In [ ]:
def decomposition(tensor, rank, method = 'tucker', rs = 1):
    tensor = tl.tensor(tensor)
    ## for rank, keep the first dimension the same if it corrosponds to number of samples, else no constraint
    if method=='tucker':
        core, factors = decomp.tucker(tensor, rank, random_state = rs)
        return core, factors
    if method == 'rank1':
        weight, factors = decomp.parafac(tensor, rank, random_state = rs)
        return weight, factors
def inner_mat(A, B, r = None, method='STM'):
    assert(A.shape==B.shape)
    M = A.shape[0]
    mat = np.ones((M,M))
    for i in range(M):
        for j in range(M):
            xi = A[i]
            xj = B[j]
            if method=='STM':
                mat[i,j] = inner_prod(xi, xj)
            if method=='SHTM':
                mat[i,j] = inner_prod(xi, xj, r)
    return mat
def inner_prod(A, B, r = None):
    if r is None:
        if A.shape == B.shape:
            return np.sum(A*B)
        if A.shape == B.shape[1:]:
            ans = []
            for b in B:
                ans.append(np.sum(A*b))
            return ans
    else:
        assert(A.shape == B.shape)
        sum = 0
        Ad = decomposition(tl.tensor(A), r, 'rank1')[1]
        Bd = decomposition(tl.tensor(B), r, 'rank1')[1]
        for p in range(r):
            for q in range(r):
                temp = 1
                for n in range(len(A.shape)):
                    temp*=np.sum(Ad[n][p]*Bd[n][q])
                sum+=temp
        return sum

In [ ]:
_ = decomposition(tl.tensor(A),1, 'rank1')[1]
np.squeeze(tl.tenalg.outer((tl.tenalg.outer((_[0],_[1])),_[2])))[1]

In [46]:
# C_weights = C*qi_prev and W_weights = w_prev both needed (similar to how minimal vc dimension tree works)
def inner_mat(A, B, M = None):
    assert(A.shape==B.shape)
    M = A.shape[0] if M is None else M
    M = min(M, A.shape[0])
    mat = np.ones((M,M))
    for i in range(M):
        for j in range(M):
            xi = A[i]
            xj = B[j]
            mat[i,j] = np.sum(xi*xj)
    return mat
def inner_prod_cp(w,x):
    return cp.sum(cp.multiply(w,x))
def inner_prod(A, B):
    if A.shape == B.shape:
        return np.sum(A*B)
    if A.shape == B.shape[1:]:
        ans = []
        for b in B:
            ans.append(np.sum(A*b))
        return ans
def solver(X, y, C = 1, method='STM', r = None, flag = True, xa = None, xb = None, methodw = 'L2', methodq = 'L1'):
    y = y.reshape(-1,1)
    assert(X.shape[0]==y.shape[0])
    if flag==False or len(X.shape)==2:
        if method == 'STM' or method == 'SHTM':
            w = np.ones(X.shape[1:])
            b = np.ones(y.shape[1:])
            M = X.shape[0]
            if method == 'STM':
                H = np.dot(y,y.T)*inner_mat(X,X)*1.0
            elif method == 'SHTM':
                r = r if r is not None else 1
                H = np.dot(y,y.T)*inner_mat(X, X, r, method='SHTM')*1.0
            P_ = cvxopt.matrix(H)
            q_ = cvxopt.matrix(-np.ones((M,1)))
            G_ = cvxopt.matrix(np.vstack((np.eye(M)*-1,np.eye(M))))
            h_ = cvxopt.matrix(np.hstack((np.zeros(M), np.ones(M)*C)))
            A_ = cvxopt.matrix(y.reshape(1,-1).astype('float'))
            b_ = cvxopt.matrix(np.zeros(1)) 
            sol = cvxopt.solvers.qp(P_, q_, G_, h_, A_, b_, options={'show_progress': False})
            alphas = np.array(sol['x'])
            w = np.tensordot((y*alphas).T, X, axes = ((1),(0))).reshape(X.shape[1:]) 
            b = -0.5*(max(inner_prod(w, X[(y==-1).reshape(-1)])) + min(inner_prod(w, X[(y==1).reshape(-1)])))
            return w, b
    else:
        #only works for greyscale right now
        xa = xa if xa is not None else np.zeros(X.shape[0])
        xb = xb if xb is not None else np.zeros(X.shape[0])
        assert(xa is not None)
        assert(xb is not None)
        assert(len(X.shape)<=3)
        if len(X.shape)==3:
            if method == 'STM' or method == 'SHTM':
                w = cp.Variable(X.shape[1:])
                b = cp.Variable((1,1))
                wa = cp.Variable((1,1))
                wb = cp.Variable((1,1))
                qi = cp.Variable(y.shape)
                if methodq == 'L1':
                    term2 = cp.sum(qi)*C
                elif methodq == 'L0':
                    term2 = 0
                    for i in range(y.shape[0]):
                        term2 += cp.maximum(0,cp.sign(qi[i]))*C
                if methodw == 'L2':
                    term1 = 1/2*cp.norm2(w)
                    term3 = 1/2*(cp.square(wa)+cp.square(wb))
                elif methodw == 'L1':
                    term1 = cp.norm1(w)
                    term3 = cp.abs(wa)+cp.abs(wb)
                elif methodw == 'L0':
                    term1 = 0
                    for i in range(X.shape[1]):
                        for j in range(X.shape[2]):
                            term1 += cp.maximum(0,cp.sign(w[i][j]))
                    term3 = cp.maximum(0,cp.sign(wa))+cp.maximum(0,cp.sign(wb))
                objfun = cp.Minimize(term1+term2+term3)
                constraints = []
                #constraints = y*(inner_prod_cp(w,X) + b + wa*xa + wb*xb) + qi >= 1
                #constraints += qi>=0
                for i in range(X.shape[0]):
                    constraints+=[cp.multiply(y[i],(inner_prod_cp(w,X[i]) + b + cp.multiply(wa,xa[i]) + cp.multiply(wb,xb[i])))+qi[i]>=1, qi[i]>=0]
                problem = cp.Problem(objfun,constraints)
                problem.solve()
                return w.value, b.value, wa.value, wb.value

In [47]:
class Node:
  def __init__(self,inp_dimensions):
    self.weight = np.zeros(inp_dimensions)
    self.bias = 0
    self.A = None
    self.B = None
    self.wA = 0
    self.wB = 0
    self.dim = inp_dimensions
    self.C1=[]
    self.C2=[]
    self.C3=[]
    self.C4=[]
    self.labels = []
    self.X = []
    self.height = 0
  def insert(self,neuron_type, weight=0, bias=0, w=0):     
    if neuron_type == 'A':
      self.A = Node(self.dim)
      self.A.weight = weight
      self.A.bias = bias
      self.A.height = self.height+1
      return self.A
    else:
      self.B = Node(self.dim)
      self.B.weight = weight
      self.B.bias = bias
      self.B.height = self.height+1
      return self.B
  def update_weights_and_bias(self,weight, bias, wA = 0, wB = 0):  
    self.weight = weight
    self.bias = bias
    self.wA = wA
    self.wB = wB
  def update_classes(self,ypred,ytrue):
    ypred=ypred.copy()
    ypred=np.reshape(ypred,(ypred.shape[0],1))    
    yf  = np.add(2*ypred, ytrue)
    self.C1 = np.argwhere(yf>2)[:,0] #1,1
    self.C2 = np.argwhere(yf<-2)[:,0] #-1,-1
    self.C3 = np.where((yf<2) & (yf>0))[0]   #1,-1
    self.C4 = np.where((yf<0) & (yf>-2))[0] #-1,1
  def forward(self, X): 
    y=[]
    X=X.copy()
    w = self.weight 
    b = self.bias
    wA = np.asarray([self.wA]).copy()
    wB=np.asarray([self.wB]).copy()
    if(self==None):
      return [] 
    if(self.A==None and self.B==None):
      y = np.sign(inner_prod(w, X)+b).reshape(-1,1)
    if(self.A==None):
      xA = np.zeros((X.shape[0],1))
    else:
      xA = self.A.forward(X)
      xA=np.reshape(xA,(xA.shape[0],1))
    if(self.B==None):
      xB = np.zeros((X.shape[0],1)) 
    else:
      xB = self.B.forward(X)
      xB=np.reshape(xB,(xB.shape[0],1))
    if(self.A!=None and self.B!=None):
      #print(b)
      #print(inner_prod(w, X))
      wA = np.asarray([np.asscalar(wA)])
      wB = np.asarray([np.asscalar(wB)])
      #print(wA)
      #print(wB)
      #print(xA)
      #print(xB)

      #print(inner_prod(wA, xA))
      #print(inner_prod(wB, xB))
      y = np.sign(inner_prod(w, X)+inner_prod(wA, xA)+inner_prod(wB, xB)+b).reshape(-1,1)
    if(self.A!=None and self.B==None):
      y = np.sign(inner_prod(w, X)+inner_prod(wA, xA)+b).reshape(-1,1)
    if(self.A==None and self.B!=None):
      y = np.sign(inner_prod(w, X)+inner_prod(wB, xB)+b).reshape(-1,1)
    return y
    
  def fine_tune_weights(self,C, methodw, methodq):
    #print("fine tune used")
    l=self.labels.copy()
    X = self.X.copy()
    xA = np.zeros((X.shape[0],1))
    xB = np.zeros((X.shape[0],1))
    #xA = 0
    #xB=0
    if(self==None):
      return   
    if(self.A!=None):
      self.A.fine_tune_weights(C, methodw=methodw, methodq=methodq)  
      xA = self.A.forward(X)
      xA=np.reshape(xA,(xA.shape[0],1))

    if(self.B!=None):
      self.B.fine_tune_weights(C, methodw=methodw, methodq=methodq) 
      xB = self.B.forward(X)
      xB=np.reshape(xB,(xB.shape[0],1))
    #print("solver started")
    weight, bias, wA, wB = solver(X,l,C,'STM',None,True,xA,xB, methodw=methodw, methodq=methodq)
    #print("ended")

    self.update_weights_and_bias(weight, bias,wA,wB)


  def recursive(self, X,labels, h,C, methodw, methodq):
    self.X = X
    self.labels = labels
    labels=labels.copy()
    X=X.copy()
    weight, bias, _, _1_ = solver(X,labels,C, methodw=methodw, methodq=methodq)
    self.update_weights_and_bias(weight, bias)
    ypred=self.forward(X)
    self.update_classes(ypred,labels)
    C1=self.C1
    C2=self.C2
    C3=self.C3
    C4=self.C4
    if(len(C3)==0 and len(C4)==0):
      return
    if(self.height>h-1):
      return
    if(len(C1)==0 or len(C2)==0):
      if(len(C1)!=0):
        x1=X[C1[0]]
        x2=X[C3[0]]
      elif(len(C2)!=0):
        x1=X[C2[0]]
        x2=X[C4[0]]
      else:
        x1=X[C3[0]]
        x2=X[C4[0]]
      weight, bias = getHyperPlaneFromTwoPoints(x1, x2)
      self.update_weights_and_bias(weight, bias)
      ypred = self.forward(X)
      self.update_classes(ypred,labels)
      C1=self.C1
      C2=self.C2
      C3=self.C3
      C4=self.C4

    if(C3 != []):
      X_new=np.take(X,np.hstack((C1,C3,C4)),axis=0)
      labels[C1]=-1
      labels[C3]=1
      labels[C4]=-1
      y_new=np.take(labels,np.hstack((C1,C3,C4)),axis=0)
      NodeA = self.insert('A')
      NodeA.recursive(X_new,y_new,h,C, methodw=methodw, methodq=methodq)
    if(C4 != []):
      X_new=np.take(X,np.hstack((C2,C3,C4)),axis=0)
      labels[C2]=-1
      labels[C3]=-1
      labels[C4]=1
      y_new=np.take(labels,np.hstack((C2,C3,C4)),axis=0)
      NodeB = self.insert('B')
      NodeB.recursive(X_new,y_new,h,C, methodw=methodw, methodq=methodq)


In [48]:
def train(X,y,height = 1000,C=1000000, methodw = 'L2', methodq = 'L1'):
  print("Start")
  Tree = Node(X.shape[1:])

  Tree.recursive(X,y,height,C, methodw=methodw, methodq=methodq)
  print("Recursion Done")

  Tree.fine_tune_weights(C, methodw=methodw, methodq=methodq)
  print("End")

  ypred=Tree.forward(X)
  print("Train accuracy:")
  print(accuracy(y,ypred))
  return Tree

In [54]:
XALL = mnist.train_images()[:5000]/255
yALL = mnist.train_labels().reshape(-1,1)[:5000].astype(np.int8)
y0 = (yALL == 0)
y2 = (yALL == 2)
ytot = y0 + y2
Xtrain = XALL[ytot.reshape(-1)]
ytrain = yALL[ytot.reshape(-1)] -1

In [56]:
model = train(Xtrain,ytrain,1,1e9,'L1','L1')

Start
Recursion Done
End
Train accuracy:
1.0


In [57]:
y=model.forward(Xtrain)
print(accuracy(y,ytrain))

1.0


In [58]:
n = Xtrain.shape[0]
i = 0
k = 5
r = int(n/5)
total_correct = 0
total_total = 0
total_accuracy = 0
while(i < k):
    print(f"{i} fold")
    if ((i+1)*r>n):
        print("lower the test %")
        break
    Xtra1 = Xtrain[0 : r*i]
    Ytra1 = ytrain[0 : r*i]
    Xtra2 = Xtrain[r*(i+1) : ]
    Ytra2 = ytrain[r*(i+1) : ]
    Xtra = np.concatenate((Xtra1, Xtra2), axis=0)
    Ytra = np.concatenate((Ytra1, Ytra2), axis=0)
    Xtest = Xtrain[r*i : r*(i+1)]
    Ytest = ytrain[r*i : r*(i+1)]
    model = train(Xtra,Ytra,1,1e9,'L1','L1')
    ypred = model.forward(Xtest)
    acc = accuracy(ypred,Ytest)
    total_accuracy += acc
    print('Test Accuracy:')
    print(acc)
    i+=1
    print('\n')
print(f"total test accuracy={total_accuracy/k}\n\n")

0 fold
Start
Recursion Done
End
Train accuracy:
1.0
Test Accuracy:
0.9689119170984456


1 fold
Start
Recursion Done
End
Train accuracy:
1.0
Test Accuracy:
0.9844559585492227


2 fold
Start
Recursion Done
End
Train accuracy:
1.0
Test Accuracy:
0.9792746113989638


3 fold
Start
Recursion Done
End
Train accuracy:
1.0
Test Accuracy:
0.9896373056994818


4 fold
Start
Recursion Done
End
Train accuracy:
1.0
Test Accuracy:
0.9844559585492227


total test accuracy=0.9813471502590673




In [18]:
XALL = mnist.train_images()[55000:]/255
yALL = mnist.train_labels().reshape(-1,1)[55000:].astype(np.int8)
y0 = (yALL == 0)
y2 = (yALL == 2)
ytot = y0 + y2
Xtrain = XALL[ytot.reshape(-1)]
ytrain = yALL[ytot.reshape(-1)] -1

In [21]:
print(Xtrain.shape)

(982, 28, 28)


1.0


In [9]:
import tensorly as tl
import tensorly.decomposition as decomp

In [10]:
def decomposition(tensor, rank, method = 'tucker', rs = 1):
    tensor = tl.tensor(tensor)
    ## for rank, keep the first dimension the same if it corrosponds to number of samples, else no constraint
    if method=='tucker':
        core, factors = decomp.tucker(tensor, rank, random_state = rs)
        return core, factors
    if method == 'rank1':
        weight, factors = decomp.parafac(tensor, rank, random_state = rs)
        return weight, factors

In [11]:
Xtrain, _ = decomposition(tl.tensor(Xtrain), [Xtrain.shape[0],20,20])


In [59]:
#from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')
#model = train(Xtrain,ytrain,3,1e9)

NameError: name 'HTML' is not defined

In [47]:
abc=np.asarray([[1],[2],[3]])
w=np.asarray([10])

In [48]:
print(w)

[10]


In [49]:
print(inner_prod(w, abc))

[10, 20, 30]
